In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
sys.path.append('/Users/bendikostvik/Desktop/')

In [ ]:
# make a function to read the data
def read_data(date, month):
    data = pd.read_csv(f'/Users/bendikostvik/Desktop/frekvensdata/2024-{month}/Taajuusdata2024-{month}-{date}.csv')
    data['Time'] = pd.to_datetime(data['Time'])
    data_min = data.resample('1T', on='Time').mean()
    return data_min

# make a function to smooth the data
def smooth_data(data):
    data_smooth = data.copy()
    data_smooth['Value'] = data_smooth['Value'].rolling(window=10).mean()
    return data_smooth

# make a function to track the frequency
def track_frequency(data):
    tracking = data.copy()

    # add a colomn for the value - 50
    tracking['relative value'] = tracking['Value'] - 50

    # add the FCR up and down values
    tracking['FCR-up'] = np.where(tracking['relative value'] < -0.1, tracking['relative value'], np.nan)
    tracking['FCR-down'] = np.where(tracking['relative value'] > 0.1, tracking['relative value'], np.nan)

    # where the FCR-up and FCR-down values are not nan, set the valure to 0.1 or -0.1
    tracking['relative value'] = np.where(tracking['FCR-up'].notna(), -0.1, tracking['relative value'])
    tracking['relative value'] = np.where(tracking['FCR-down'].notna(), 0.1, tracking['relative value'])

    # add the integrated value to the data
    tracking['relative power'] = tracking['relative value'] * (-1)/60*10 # (MW) 1 is the bidding power
    tracking['integrated energy'] = (-1)*tracking['relative power'].cumsum() # (MWh) relative to the bidding power

    # find local maxima and minima for the integrated value
    tracking['charging'] = np.where((tracking['integrated energy'].shift(1) > tracking['integrated energy']) & (tracking['integrated energy'].shift(-1) < tracking['integrated energy']), tracking['integrated energy'], np.nan)
    tracking['discharging'] = np.where((tracking['integrated energy'].shift(1) < tracking['integrated energy']) & (tracking['integrated energy'].shift(-1) > tracking['integrated energy']), tracking['integrated energy'], np.nan)

    return tracking


In [ ]:
from optimering_FCR_N_og_D_opp import run_model_dag as run_basis

from optimering_timevis import run_model_dag as run_uten_vann

from optimering_timevis_vann_SOC_sensitiv_NogD_halv_prisD import run_model_dag as run_model_dag_halvprisD

from optimering_timevis_vann_SOC_sensitiv import run_model_dag as run_model_dag_kun_N

from optimering_timevis_vann_SOC_sensitiv_NogD import run_model_dag as run_model_dag_fullprisD

In [ ]:
# Beregner deltakelse ut ifra frekvensavvik med et internt effekttap på 7%
def aktivering(frekvensavvik):
    if -0.1 < frekvensavvik < 0.1:
        return frekvensavvik * 10
    
    elif frekvensavvik < -0.1:
        return -0.1 * 10 * 1.07
    
    else:
        return 0.1 * 10 * 1.07

# definere om batteriet deltar i markedet eller ikke
def deltakelse(markedspris, budpris):
    if budpris <= markedspris:
        return 1
    else:
        return 0
    
# Beregne inntekt per minutt
def inntekt_funksjon(markedspris, budpris, volum):
    deltakelse_value = deltakelse(markedspris, budpris) # [0 eller 1]
    return markedspris * deltakelse_value * volum / 60 # [EUR]/h

# Beregne kostnad per minutt
def kostnad_funksjon(volum, installert_kapasitet, aktivering, investeringskostnad, markedspris, budpris):
    kalendertap = 1/(20*365) # Beregner verditapet uten særlig aktivering med antakelse om levetid på 20år
    syklustap_value = syklustapet(volum, installert_kapasitet, aktivering) # [MWh/MWh]
    deltakelse_value = deltakelse(markedspris, budpris) # [0 eller 1]
    return (deltakelse_value * syklustap_value / 0.2 + kalendertap / (24*60)) * investeringskostnad # [EUR]


def investeringskostnad_funk(installert_kapasitet):
    return 500000 * installert_kapasitet # [EUR]

# Beregner C-rate
def C_raten(volum, installert_kapasitet, aktivering):
    return aktivering * 1.07 * volum / installert_kapasitet  # [h^-1]

def ladningen(volum, aktivering):
    spenning = 1000 # [V]
    return volum * aktivering * 1.07 / spenning # [Ah]

def syklustapet(volum, installert_kapasitet, aktivering):
    DOD_faktor = 0.0008 # [MWh/MWh]
    C_faktor = 0.3903 # [h]
    ladning_value = ladningen(volum, aktivering) # [Ah]
    C_rate_value = C_raten(volum, installert_kapasitet, aktivering) # [h^-1]
    return ladning_value * aktivering * DOD_faktor * np.exp(C_faktor * C_rate_value) # [MWh/MWh]

# Tar daglige tap og investeringskostnad som input for å beregne levetid
def levetid(pris_i_k, installert_kapasitet, kostnad, dag = True):
    if dag:
        levetid_verdi = pris_i_k * installert_kapasitet / (365 * kostnad) # [år] Overfører daglige tap til årlig tap
    else:
        levetid_verdi = pris_i_k * installert_kapasitet / kostnad # [år]
    return levetid_verdi


# Henter levetid og tar daglig inntekt som input for å beregne NNV
def NNV_investering(Inntekt, kostnad, installert_kapasitet, pris_i_k = 500000, rente = 0.05, dag = True, months = None):
    if months is None:
        levetid_verdi = levetid(pris_i_k, installert_kapasitet, kostnad, dag) # [år]

        NNV = 0 # [EUR]
        if dag:
            for i in np.arange(0, int(levetid_verdi), 1): 
                NNV += (Inntekt * 365) / ((1 + rente) ** i) # [EUR] Beregner NNV for hvert år
            # Legger til det som er igjen av inntekten i den siste måneden
            NNV += (levetid_verdi % 1) * (Inntekt * 365) / ((1 + rente) ** (levetid_verdi+1)) # [EUR]
        
        else:
            for i in np.arange(0, int(levetid_verdi), 1):
                NNV += (Inntekt) / ((1 + rente) ** i) # [EUR] Beregner NNV for hvert år
            # Legger til det som er igjen av inntekten i den siste måneden
            NNV += (levetid_verdi % 1) * (Inntekt) / ((1 + rente) ** (levetid_verdi+1)) # [EUR]

        
    else:
        Inntekt = Inntekt * (1 + (12 - months) / 12) # [EUR] Beregner hva inntekten ville vært om disse månedene var hele året
        kostnad = kostnad * (1 + (12 - months) / 12) # [EUR] Beregner hva kostnaden ville vært om disse månedene var hele året

        levetid_verdi = levetid(pris_i_k, installert_kapasitet, kostnad, dag) # [år]

        NNV = 0 # [EUR]
        for i in np.arange(0, int(levetid_verdi), 1):
            NNV += (Inntekt) / ((1 + rente) ** i) # [EUR] Beregner NNV for hvert år
        # Legger til det som er igjen av inntekten i den siste måneden
        NNV += (levetid_verdi % 1) * (Inntekt) / ((1 + rente) ** (levetid_verdi+1)) # [EUR]

    return NNV - installert_kapasitet * pris_i_k # [EUR]



def eval_optimeringsresultat_data(markedspris, frekvensavvik, budpris, volum, installert_kapasitet):
    aktivering_value = aktivering(frekvensavvik)
    investeringskostnad_value = investeringskostnad_funk(installert_kapasitet)

    inntekt_value = inntekt_funksjon(markedspris, budpris, volum)
    kostnad_value = kostnad_funksjon(volum, installert_kapasitet, aktivering_value, investeringskostnad_value, markedspris, budpris)

    return inntekt_value, kostnad_value



# Beregninger

In [ ]:
# read /Users/bendikostvik/Desktop/activation_year.csv
df_year = pd.read_csv('/Users/bendikostvik/Desktop/activation_year.csv', sep=';')
# set the name Time
df_year = df_year.rename(columns={'Unnamed: 0': 'Time'})
# name the first column time
df_year['Time'] = pd.to_datetime(df_year['Time'])

In [ ]:
# lag df for april
april_2024 = df_year[(df_year['Time'] >= '2024-04-01') & (df_year['Time'] < '2024-05-01')]

# lag df for mai
may_2024 = df_year[(df_year['Time'] >= '2024-05-01') & (df_year['Time'] < '2024-06-01')]
may_2024_copy = may_2024.copy()
may_2024_u_vann = may_2024.copy()

# lag df for juni
june_2024 = df_year[(df_year['Time'] >= '2024-06-01') & (df_year['Time'] < '2024-06-06')]
june_2024_fullprisD = june_2024.copy()
june_2024_halvprisD = june_2024.copy()
june_2024_u_vann = june_2024.copy()



In [ ]:
# kjør basismodellen for april, mai og juni
april_results = run_basis(april_2024, csv = False)
may_results = run_basis(may_2024, csv = False)
june_results = run_basis(june_2024, csv = False)

In [ ]:
may_results_1a = run_uten_vann(may_2024_u_vann, csv = False)
may_results_kun_N = run_model_dag_kun_N(may_2024_copy, csv = False)


june_results_1a = run_uten_vann(june_2024_u_vann, csv = False)

In [ ]:
june_results_fullprisD = run_model_dag_fullprisD(june_2024_fullprisD, csv = False)
june_results_halvprisD = run_model_dag_halvprisD(june_2024_halvprisD, csv = False)

# Resultater

## Månedsvis

In [ ]:
# plot Optimal Budvolum and Optimal Budvolum opp
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=june_results_1a['Time'], 
                         y=june_results_1a['Optimal Budvolum opp'], 
                         mode='lines', name='FCR-D opp, kun Batteri'), 
                         secondary_y=False)

#fig.add_trace(go.Scatter(x=june_results['Time'], y=june_results['Optimal Budvolum opp'] + june_results['Optimal Budvolum'], mode='lines', name='Optimal Budvolum opp + Optimal Budvolum'), secondary_y=False)
fig.add_hline(y=0, line_dash="dot", line_color="black")
fig.update_layout( xaxis_title='Time', yaxis_title='[MW]')


fig.add_trace(go.Scatter(x=june_results['Time'], 
                        y=june_results['Optimal Budvolum opp'], 
                        mode='lines', 
                        name='FCR-D opp, Batteri + Vann'), 
                        secondary_y=False)




fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=may_results_kun_N['Time'], y=may_results_kun_N['SOC'], mode='lines', name='SOC (Kun FCR-N)'))
fig.add_trace(go.Scatter(x=may_results['Time'], y=may_results['SOC'], mode='lines', name='SOC (FCR-N + FCR-D)'))
fig.update_layout(xaxis_title='Dato', yaxis_title='SOC [andel]')

In [ ]:
# plot inntekt total
fig = go.Figure()
fig.add_trace(go.Scatter(x=may_results_kun_N['Time'], y=may_results_kun_N['Inntekt Total'], mode='lines', name='Inntekt (Kun FCR-N)'))
fig.add_trace(go.Scatter(x=may_results['Time'], y=may_results['Inntekt Total'], mode='lines', name='Inntekt (FCR-N + FCR-D)'))
fig.update_layout(xaxis_title='Dato', yaxis_title='[EUR]')

fig.show()

# plot kostnad batteri
fig = go.Figure()
fig.add_trace(go.Scatter(x=may_results_kun_N['Time'], y=may_results_kun_N['Kostnad Batteri'], mode='lines', name='Kostnad Batteri (Kun FCR-N)'))
fig.add_trace(go.Scatter(x=may_results['Time'], y=may_results['Kostnad Batteri'], mode='lines', name='Kostnad Batteri (FCR-N + FCR-D)'))
fig.update_xaxes(title_text='Dato')
fig.update_yaxes(title_text='[EUR]')
fig.show()

In [ ]:
print('Inntekt Total (Kun FCR-N):', may_results_kun_N['Inntekt Total'].sum())
print('Inntekt Total (FCR-N + FCR-D):', may_results['Inntekt Total'].sum())
print('Kostnad Batteri (Kun FCR-N):', may_results_kun_N['Kostnad Batteri'].sum())
print('Kostnad Batteri (FCR-N + FCR-D):', may_results['Kostnad Batteri'].sum())

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=may_results_1a['Time'], y=may_results_1a['SOC'], mode='lines', name='SOC Batteri'))
fig.add_trace(go.Scatter(x=may_results['Time'], y=may_results['SOC'], mode='lines', name='SOC Batteri + Vann'))
fig.update_layout(xaxis_title='Dato', yaxis_title='SOC [MWh/MWh]')
fig.show()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot Optimal Budvolum for june_results_fullprisD
fig.add_trace(go.Scatter(x=june_results_fullprisD['Time'], 
                         y=june_results_fullprisD['Optimal Budvolum'], 
                         mode='lines', 
                         name='2024-pris FCR-D'), 
              secondary_y=False)

# Plot Optimal Budvolum for june_results
fig.add_trace(go.Scatter(x=june_results_halvprisD['Time'], 
                         y=june_results_halvprisD['Optimal Budvolum'], 
                         mode='lines', 
                         name='Halv pris FCR-D'), 
              secondary_y=False)

# Add a horizontal line at y=0
fig.add_hline(y=0, line_dash="dot", line_color="black")

# Update layout
fig.update_layout(
                  xaxis_title='Time', 
                  yaxis_title='[MW]')
fig.update_yaxes(title_text="MW", secondary_y=True)

fig.show()

In [ ]:
print(june_results_halvprisD['Optimal Budvolum'].sum())
print(june_results_fullprisD['Optimal Budvolum'].sum())
print(june_results_halvprisD['Optimal Budvolum opp'].sum())
print(june_results_fullprisD['Optimal Budvolum opp'].sum())

## Dag for Dag

### base

In [ ]:
# loop over every day from the first of january 2024 to the 29th of september 2024
# and run the model for each day
monts = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']
inntekt_dag = []
datolist = []
kostnad_dag = []
kostnad_dag_vann = []
aktivitet_dag = []
pris_dag = []

for month in monts:
    for day in days:

        day_df = df_year[(df_year['Time'] >= f'2024-{month}-{day}') & (df_year['Time'] < f'2024-{month}-{int(day)+1:02d}')]
        # run the model for each day

        results = run_basis(day_df, csv = False)
        # add the results to the list
        inntekt_dag.append(results['Inntekt Total'].sum())
        kostnad_dag.append(results['Kostnad Batteri'].sum())
        kostnad_dag_vann.append(results['Kostnad Vann'].sum())
        aktivitet_dag.append(results['Optimal Budvolum'].sum() + results['Optimal Deltakelse Vann'].sum() * 0.7)
        pris_dag.append(results['Pris'].mean())


        # print the date and the income
        datolist.append(f'2024-{month}-{day}')

In [ ]:
inntekt_results = pd.DataFrame({'Date': datolist, 'Inntekt': inntekt_dag, 'Kostnad': kostnad_dag, 'Kostnad Vann': kostnad_dag_vann, 'Aktivitet': aktivitet_dag, 'Pris FCR-N': pris_dag})
inntekt_results['Date'] = pd.to_datetime(inntekt_results['Date'])


In [ ]:
# what day has the highest income
highest_income = inntekt_results.loc[inntekt_results['Inntekt'].idxmax()]
print(f'Highest income: {highest_income["Inntekt"]} on {highest_income["Date"]}')

In [ ]:
# what day hast the highest cost
highest_cost = inntekt_results.loc[inntekt_results['Kostnad'].idxmax()]
print(f'Highest cost: {highest_cost["Kostnad"]} on {highest_cost["Date"]}')

# what was the income on the 1st of may
may_1_income = inntekt_results.loc[inntekt_results['Date'] == '2024-05-01']
print(f'Income on 1st of May: {may_1_income["Inntekt"].values[0]}')

In [ ]:
# make a month dictionary where the key is the month number and the value is the month name in norwegian
month_dict = {
    1: 'Januar',
    2: 'Februar',
    3: 'Mars',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Desember'
}
# Group by month and calculate mean and standard deviation for each metric
monthly_stats = inntekt_results.groupby(inntekt_results['Date'].dt.month).agg(
    mean_kostnad=('Kostnad', 'mean'),
    std_kostnad=('Kostnad', 'std'),
    mean_inntekt=('Inntekt', 'mean'),
    std_inntekt=('Inntekt', 'std'),
    mean_aktivitet=('Aktivitet', 'mean'),
    std_aktivitet=('Aktivitet', 'std')
)

# Add month names to the index
monthly_stats.index = monthly_stats.index.map(month_dict)

# Extract mean and standard deviation values from monthly_stats
mean_inntekt = monthly_stats['mean_inntekt'].tolist()
std_inntekt = monthly_stats['std_inntekt'].tolist()
mean_kostnad = monthly_stats['mean_kostnad'].tolist()
std_kostnad = monthly_stats['std_kostnad'].tolist()
mean_aktivitet = monthly_stats['mean_aktivitet'].tolist()
std_aktivitet = monthly_stats['std_aktivitet'].tolist()

# Extract month names for x-axis
months = monthly_stats.index.tolist()


In [ ]:
# Calculate Inntekt ± Std for all months
inntekt_std_plus = [mean_inntekt[i] + std_inntekt[i] for i in range(len(mean_inntekt))]
inntekt_std_minus = [mean_inntekt[i] - std_inntekt[i] for i in range(len(mean_inntekt))]

# Plot Inntekt with Std for all months
fig = go.Figure()
fig.add_trace(go.Scatter(x=months, y=mean_inntekt, mode='lines+markers', name='Inntekt'))
fig.add_trace(go.Scatter(
    x=months, 
    y=inntekt_std_plus, 
    mode='lines', 
    name='Inntekt + Std', 
    line=dict(color='lightblue'),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=months, 
    y=inntekt_std_minus, 
    mode='lines', 
    name='Standardavvik', 
    line=dict(color='lightblue'),
    fill='tonexty', 
    fillcolor='rgba(173, 216, 230, 0.3)'  # Light blue with transparency
))
fig.update_layout(xaxis_title='Måned', yaxis_title='[EUR]')
fig.show()

# Calculate Kostnad ± Std for all months
kostnad_std_plus = [mean_kostnad[i] + std_kostnad[i] for i in range(len(mean_kostnad))]
kostnad_std_minus = [mean_kostnad[i] - std_kostnad[i] for i in range(len(mean_kostnad))]

# Plot Kostnad with Std for all months
fig = go.Figure()
fig.add_trace(go.Scatter(x=months, y=mean_kostnad, mode='lines+markers', name='Kostnad'))
fig.add_trace(go.Scatter(
    x=months, 
    y=kostnad_std_plus, 
    mode='lines', 
    name='Kostnad + Std', 
    line=dict(color='lightblue'),
    showlegend=False
))
fig.add_trace(go.Scatter(
    x=months, 
    y=kostnad_std_minus, 
    mode='lines', 
    name='Standardavvik', 
    line=dict(color='lightblue'),
    fill='tonexty', 
    fillcolor='rgba(173, 216, 230, 0.3)'  # Light blue with transparency
))
fig.update_layout(xaxis_title='Måned', yaxis_title='[EUR]')
fig.show()

In [ ]:
# remoove december
inntekt_results = inntekt_results[inntekt_results['Date'] < '2024-12-01']


# lag en dictionary for måneder og farger
month_colors = {
    1: 'blue',
    2: 'green',
    3: 'purple',
    4: 'red',
    5: 'orange',
    6: 'pink',
    7: 'brown',
    8: 'magenta',
    9: 'gray',
    10: 'cyan',
    11: 'yellow'
}




inntekt_results['month'] = inntekt_results['Date'].dt.month
# scatter plot with pris and aktivitet
fig = go.Figure()
for month in inntekt_results['month'].unique():
    month_data = inntekt_results[inntekt_results['month'] == month]
    fig.add_trace(go.Scatter(
        x=month_data['Pris FCR-N'], 
        y=month_data['Aktivitet'], 
        mode='markers', 
        marker=dict(color=month_colors[month], size=7),
        name=f'{month_dict[month]}'
    ))

# Add a horizontal line at y=15
fig.add_hline(y=15, line_dash="dot", line_color="black")
# Add a vertical line at x=20
fig.add_vline(x=20, line_dash="dot", line_color="black")
fig.update_layout(xaxis_title='Pris FCR-N [EUR]', yaxis_title='Integrert Deltakelse FCR-N [MW*h]')
fig.show()

In [ ]:
# count the number of days each month where the price is above 20
months_over_20 = []
for month in inntekt_results['month'].unique():
    month_data = inntekt_results[inntekt_results['month'] == month]
    count = month_data[month_data['Aktivitet'] > 20].shape[0]
    months_over_20.append(count)

# and for 15
months_over_15 = []
for month in inntekt_results['month'].unique():
    month_data = inntekt_results[inntekt_results['month'] == month]
    count = month_data[month_data['Aktivitet'] > 15].shape[0]
    months_over_15.append(count)

print(months_over_20)
print(months_over_15)

print((15+8+9+16)/sum(months_over_20))
print((19+14+13+21)/sum(months_over_15))

### uten vann

In [ ]:
inntekt_u_vann = []
kostnad_u_vann = []
datolist = []

for month in monts:
    for day in days:

        day_df = df_year[(df_year['Time'] >= f'2024-{month}-{day}') & (df_year['Time'] < f'2024-{month}-{int(day)+1:02d}')]
        # run the model for each day

        results = run_uten_vann(day_df, csv = False)
        # add the results to the list
        inntekt_u_vann.append(results['Inntekt Total'].sum())
        kostnad_u_vann.append(results['Kostnad Batteri'].sum())

        # print the date and the income
        datolist.append(f'2024-{month}-{day}')
# create a dataframe with the results   


In [ ]:
df_results_u_vann = pd.DataFrame({'Date': datolist, 'Inntekt': inntekt_u_vann, 'Kostnad': kostnad_u_vann})

df_results_u_vann['Date'] = pd.to_datetime(df_results_u_vann['Date'])

In [ ]:
# halvprisD
inntekt_halvprisD = []
kostnad_halvprisD = []
datolist = []
for month in monts:
    for day in days:

        day_df = df_year[(df_year['Time'] >= f'2024-{month}-{day}') & (df_year['Time'] < f'2024-{month}-{int(day)+1:02d}')]
        # run the model for each day

        results = run_model_dag_halvprisD(day_df, csv = False)
        # add the results to the list
        inntekt_halvprisD.append(results['Inntekt Total'].sum())
        kostnad_halvprisD.append(results['Kostnad Batteri'].sum())

        # print the date and the income
        datolist.append(f'2024-{month}-{day}')
# create a dataframe with the results
df_results_halvprisD = pd.DataFrame({'Date': datolist, 'Inntekt': inntekt_halvprisD, 'Kostnad': kostnad_halvprisD})
df_results_halvprisD['Date'] = pd.to_datetime(df_results_halvprisD['Date'])
# fullprisD
inntekt_fullprisD = []
kostnad_fullprisD = []
datolist = []
for month in monts:
    for day in days:

        day_df = df_year[(df_year['Time'] >= f'2024-{month}-{day}') & (df_year['Time'] < f'2024-{month}-{int(day)+1:02d}')]
        # run the model for each day

        results = run_model_dag_fullprisD(day_df, csv = False)
        # add the results to the list
        inntekt_fullprisD.append(results['Inntekt Total'].sum())
        kostnad_fullprisD.append(results['Kostnad Batteri'].sum())

        # print the date and the income
        datolist.append(f'2024-{month}-{day}')
# create a dataframe with the results
df_results_fullprisD = pd.DataFrame({'Date': datolist, 'Inntekt': inntekt_fullprisD, 'Kostnad': kostnad_fullprisD})
df_results_fullprisD['Date'] = pd.to_datetime(df_results_fullprisD['Date'])
# remove december
df_results_fullprisD = df_results_fullprisD[df_results_fullprisD['Date'] < '2024-12-01']
df_results_halvprisD = df_results_halvprisD[df_results_halvprisD['Date'] < '2024-12-01']
df_results_u_vann = df_results_u_vann[df_results_u_vann['Date'] < '2024-12-01']

In [ ]:
# kun FCR-N
inntekt_kun_N = []
kostnad_kun_N = []
datolist = []
for month in monts:
    for day in days:

        day_df = df_year[(df_year['Time'] >= f'2024-{month}-{day}') & (df_year['Time'] < f'2024-{month}-{int(day)+1:02d}')]
        # run the model for each day

        results = run_model_dag_kun_N(day_df, csv = False)
        # add the results to the list
        inntekt_kun_N.append(results['Inntekt Total'].sum())
        kostnad_kun_N.append(results['Kostnad Batteri'].sum())

        # print the date and the income
        datolist.append(f'2024-{month}-{day}')
# create a dataframe with the results
df_results_kun_N = pd.DataFrame({'Date': datolist, 'Inntekt': inntekt_kun_N, 'Kostnad': kostnad_kun_N})
df_results_kun_N['Date'] = pd.to_datetime(df_results_kun_N['Date'])
# remove december
df_results_kun_N = df_results_kun_N[df_results_kun_N['Date'] < '2024-12-01']

In [ ]:
# print the sum of inntekt and kostnad for all models
print('Inntekt FCR-N + FCR-D ned fullpris: ', df_results_fullprisD['Inntekt'].sum())
print('Kostnad FCR-N + FCR-D ned fullpris: ', df_results_fullprisD['Kostnad'].sum())
print('Inntekt FCR-N + FCR-D ned halvpris: ', df_results_halvprisD['Inntekt'].sum())
print('Kostnad FCR-N + FCR-D ned halvpris: ', df_results_halvprisD['Kostnad'].sum())
print('Inntekt FCR-N + FCR-D uten vann: ', df_results_u_vann['Inntekt'].sum())
print('Kostnad FCR-N + FCR-D uten vann: ', df_results_u_vann['Kostnad'].sum())
print('Inntekt FCR-N: ', df_results_kun_N['Inntekt'].sum())
print('Kostnad FCR-N: ', df_results_kun_N['Kostnad'].sum())
print('base inntekt: ', inntekt_results['Inntekt'].sum())
print('base kostnad: ', inntekt_results['Kostnad'].sum())


In [ ]:
# for each month calculate the mean cost and income for each model
mean_cost_base = inntekt_results.groupby(inntekt_results['Date'].dt.month)['Kostnad'].mean()
mean_income_base = inntekt_results.groupby(inntekt_results['Date'].dt.month)['Inntekt'].mean()

# multiply each month by the number of days in the month
mean_cost_base = mean_cost_base * [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]
mean_income_base = mean_income_base * [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]

# make december the mean of november and january
kostnad_des = (mean_cost_base[11] + mean_cost_base[1]) / 2
inntekt_des = (mean_income_base[11] + mean_income_base[1]) / 2

print(mean_income_base.sum() + inntekt_des , mean_cost_base.sum() + kostnad_des)

In [ ]:
NNV_investering(160650, 90637, 2, dag=False, rente = 0.05)

In [ ]:
NNV_investering(160650, 90637, 2, dag = False, rente=0.10)

In [ ]:
levetid(500000, 2, 90637, dag=False)

In [ ]:
# do the same for uten vann
mean_cost_u_vann = df_results_u_vann.groupby(df_results_u_vann['Date'].dt.month)['Kostnad'].mean()
mean_income_u_vann = df_results_u_vann.groupby(df_results_u_vann['Date'].dt.month)['Inntekt'].mean()
# multiply each month by the number of days in the month
mean_cost_u_vann = mean_cost_u_vann * [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]
mean_income_u_vann = mean_income_u_vann * [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]
# make december the mean of november and january
kostnad_des = (mean_cost_u_vann[11] + mean_cost_u_vann[1]) / 2
inntekt_des = (mean_income_u_vann[11] + mean_income_u_vann[1]) / 2
print(mean_income_u_vann.sum() + inntekt_des , mean_cost_u_vann.sum() + kostnad_des)
NNV_investering(mean_income_u_vann.sum() + inntekt_des , mean_cost_u_vann.sum() + kostnad_des, 2, rente=0.05)

In [ ]:
NNV_investering(mean_income_u_vann.sum() + inntekt_des , mean_cost_u_vann.sum() + kostnad_des, 2, dag=False, rente=0.1)

In [ ]:
levetid(500000, 2, mean_cost_u_vann.sum() + kostnad_des, dag=False)

In [ ]:
# do the same for halvert
mean_cost_halvprisD = df_results_halvprisD.groupby(df_results_halvprisD['Date'].dt.month)['Kostnad'].mean()
mean_income_halvprisD = df_results_halvprisD.groupby(df_results_halvprisD['Date'].dt.month)['Inntekt'].mean()
# multiply each month by the number of days in the month
mean_cost_halvprisD = mean_cost_halvprisD * [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]
mean_income_halvprisD = mean_income_halvprisD * [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30]
# make december the mean of november and january
kostnad_des = (mean_cost_halvprisD[11] + mean_cost_halvprisD[1]) / 2
inntekt_des = (mean_income_halvprisD[11] + mean_income_halvprisD[1]) / 2
print(mean_income_halvprisD.sum() + inntekt_des , mean_cost_halvprisD.sum() + kostnad_des)
NNV_investering(mean_income_halvprisD.sum() + inntekt_des , mean_cost_halvprisD.sum() + kostnad_des, 2, rente=0.05)

In [ ]:
levetid(500000, 2, 91213.9, dag=False)

In [ ]:
# do the same, but only for the summer months
# summer months are april, mai, juni, juli, august, september
# create a list of summer months
# Filter the DataFrame for summer months and extract the corresponding values
summer_filter_kun_N = (df_results_kun_N['Date'] >= '2024-04-01') & (df_results_kun_N['Date'] < '2024-10-01')
inntekt_kun_N_summer = df_results_kun_N.loc[summer_filter_kun_N, 'Inntekt'].tolist()
kostnad_kun_N_summer = df_results_kun_N.loc[summer_filter_kun_N, 'Kostnad'].tolist()

summer_filter_fullprisD = (df_results_fullprisD['Date'] >= '2024-04-01') & (df_results_fullprisD['Date'] < '2024-10-01')
inntekt_fullprisD_summer = df_results_fullprisD.loc[summer_filter_fullprisD, 'Inntekt'].tolist()
kostnad_fullprisD_summer = df_results_fullprisD.loc[summer_filter_fullprisD, 'Kostnad'].tolist()

summer_filter_halvprisD = (df_results_halvprisD['Date'] >= '2024-04-01') & (df_results_halvprisD['Date'] < '2024-10-01')
inntekt_halvprisD_summer = df_results_halvprisD.loc[summer_filter_halvprisD, 'Inntekt'].tolist()
kostnad_halvprisD_summer = df_results_halvprisD.loc[summer_filter_halvprisD, 'Kostnad'].tolist()

summer_filter_u_vann = (df_results_u_vann['Date'] >= '2024-04-01') & (df_results_u_vann['Date'] < '2024-10-01')
inntekt_u_vann_summer = df_results_u_vann.loc[summer_filter_u_vann, 'Inntekt'].tolist()
kostnad_u_vann_summer = df_results_u_vann.loc[summer_filter_u_vann, 'Kostnad'].tolist()

summer_filter_base = (inntekt_results['Date'] >= '2024-04-01') & (inntekt_results['Date'] < '2024-10-01')
inntekt_base_summer = inntekt_results.loc[summer_filter_base, 'Inntekt'].tolist()
kostnad_base_summer = inntekt_results.loc[summer_filter_base, 'Kostnad'].tolist()
# print the sum of inntekt and kostnad for all models
print('Inntekt FCR-N + FCR-D ned fullpris summer: ', sum(inntekt_fullprisD_summer))
print('Kostnad FCR-N + FCR-D ned fullpris summer: ', sum(kostnad_fullprisD_summer))
print('Inntekt FCR-N + FCR-D ned halvpris summer: ', sum(inntekt_halvprisD_summer))
print('Kostnad FCR-N + FCR-D ned halvpris summer: ', sum(kostnad_halvprisD_summer))
print('Inntekt FCR-N + FCR-D uten vann summer: ', sum(inntekt_u_vann_summer))
print('Kostnad FCR-N + FCR-D uten vann summer: ', sum(kostnad_u_vann_summer))
print('Inntekt FCR-N summer: ', sum(inntekt_kun_N_summer))
print('Kostnad FCR-N summer: ', sum(kostnad_kun_N_summer))
print('base inntekt summer: ', sum(inntekt_base_summer))
print('base kostnad summer: ', sum(kostnad_base_summer))

In [ ]:

# Read data
frec_08_09 = pd.read_csv('/Users/bendikostvik/Desktop/frekvensdata/2024-08/Taajuusdata2024-08-09.csv')

# transform the 'Time' column to datetime
frec_08_09['Time'] = pd.to_datetime(frec_08_09['Time'])

# resample the data to 1 minute intervals
frec_08_09_min = frec_08_09.resample('1T', on='Time').mean()
# add a mooving avarage filter to the plot
frec_08_09_min_smooth = frec_08_09_min.copy()
frec_08_09_min_smooth['Value'] = frec_08_09_min_smooth['Value'].rolling(window=10).mean()

tracking_8_09 = frec_08_09_min_smooth.copy()
tracking_8_09['relative value'] = tracking_8_09['Value'] - 50

# add the integrated value to the data
tracking_8_09['relative effekt'] = tracking_8_09['relative value'] * 10
tracking_8_09['relative energy momentan'] = tracking_8_09['relative effekt'] * (1)/60
tracking_8_09['integrated value'] = tracking_8_09['relative energy momentan'].cumsum()

# plot the integrated value and relative value using subplots with different y-axes
fig = go.Figure()

# Add integrated value to the primary y-axis
fig.add_trace(go.Scatter(x=tracking_8_09.index, y=tracking_8_09['integrated value'], mode='lines', name='Energi'))

# Add relative value to the secondary y-axis
fig.add_trace(go.Scatter(x=tracking_8_09.index, y=tracking_8_09['relative effekt'], mode='lines', name='Effekt'))

# Add a horizontal line at y=0 for both y-axes
fig.add_hline(y=0, line_dash="dot", line_color="black")
#fig.add_hline(y=0, line_dash="dot", line_color="black", secondary_y=True)

# Update layout
fig.update_layout(xaxis_title="Time")
fig.update_yaxes(title_text="MW, MWh")


fig.show()

In [ ]:
opp_ned_og_saann = track_frequency(frec_08_09_min)
# plot the fcr up and down
fig = go.Figure()

# Add relative value line
fig.add_trace(go.Scatter(x=opp_ned_og_saann.index, y=opp_ned_og_saann['relative value'], mode='lines', name='Respons FCR-N'))

# Add FCR-up markers
fig.add_trace(go.Scatter(x=opp_ned_og_saann.index, y=opp_ned_og_saann['FCR-up'], mode='markers', name='FCR-D opp'))

# Add FCR-down markers
fig.add_trace(go.Scatter(x=opp_ned_og_saann.index, y=opp_ned_og_saann['FCR-down'], mode='markers', name='FCR-D ned'))

# Add a horizontal line at y=0
fig.add_hline(y=0, line_dash="dot", line_color="black")

# Update layout
fig.update_layout(xaxis_title='Tid', yaxis_title='Frekvensavvik')

fig.show()